In [59]:
import xmltodict
import pandas as pd
import os
"""
	El código toma el archivo recibido de palo alto como xml y lo 
	convierte a un archivo JSON que puede ser leído de manera más 
	sencilla por el programa de mapa de ataques o cualquier otro que
	consuma la info por JS
"""

'\n\tEl código toma el archivo recibido de palo alto como xml y lo \n\tconvierte a un archivo JSON que puede ser leído de manera más \n\tsencilla por el programa de mapa de ataques o cualquier otro que\n\tconsuma la info por JS\n'

In [106]:
home = os.path.expanduser('~')

color_code = {'critical':'red',
              'high':'orange',
             'medium':'yellow',
             'low':'green',
             'informational':'blue'}
default_country = 'MX'

In [107]:
with open('{}/NorsePi/XML/LastHour.xml'.format(home)) as fd:
	doc = xmltodict.parse(fd.read())

reform = {(outerKey, innerKey): values for outerKey, innerDict in doc.items() for innerKey, values in innerDict.items()}
a = pd.DataFrame(reform)
reform = a['response']['result']['log']
a = pd.DataFrame(reform)
total = a['logs']['@count']
reform = a['logs']['entry']
df = pd.DataFrame(reform)
#df = df.drop_duplicates(subset=['threatid','src']).reset_index(drop=True)

In [108]:
df = df[['direction','device_name','time_generated',"src",'srcloc','dst','dstloc','subtype','threatid','severity']]
df1 = df['device_name'].map(lambda x : x.split('-')[1])

In [109]:
countries = pd.read_csv(os.path.expanduser('~/NorsePi/CSV/all_countries.csv'),sep='\t',index_col=0)
# countries['country'] = countries['country'].map(lambda x: x.strip())

In [110]:
Tec = pd.read_csv(os.path.expanduser('~/NorsePi/CSV/GPSTec.csv'))

In [111]:
df['srcname'] = ''
df['srclat'] = ''
df['srclong'] = ''
df['dstname'] = ''
df['dstlat'] = ''
df['dstlong'] = ''
a = len(df)
for idx in range(a):
    print('{}/{}'.format(idx+1,a),end='\r')
    
    """
    Pone el codigo alpha-2 como identificador de source y destiny
    """
    df['dstloc'][idx] = df['device_name'][idx].split('-')[1]
    df['srcloc'][idx] = df['srcloc'][idx]['@cc']
    
    """
    Si viene como rango de IP, cambia por el default ('MX')
    
    Checar aqui las ubicaciones del TEC!!!!
    
    """
#     if len(df['dstloc'][idx]) > 2:
#         #print('{}:{}->"MX"'.format(idx,df['dstloc'][idx]))
#         df['dstloc'][idx] = df['device_name'][idx].split('-')[1]
    if len(df['srcloc'][idx]) > 2:
        #print('{}->"MX"'.format(df['srcloc'][idx]))
        df['srcloc'][idx] = default_country ######################################3
    
    """
    Cambia severidad por codigo de colores
    """
    df['severity'][idx] = color_code[df['severity'][idx]]
    
    """
    Cambia el nombre del pais de acuerdo al codigo alpha-2
    """
    try:
        df['srcname'][idx] = countries[countries['country'] == df['srcloc'][idx]]['name'].reset_index(drop=True)[0]
    except:
        print('Pais no encontrado: alpha2={}, indice={}'.format(df['srcloc'][idx],idx))
        df['srcloc'][idx] = default_country
        df['srcname'][idx] = 'INTERNO'
        
    """Ideia: Botar uma flag em caso de que nao seja pais atacado! :D"""
    tmp = Tec[Tec['Campus'] == df['dstloc'][idx]]['Nombre'] #!!!!!!!!!!!!!!!!!
    if len(tmp) == 0:
        print('Campus no encontrado: {}, indice={}'.format(df['dstloc'][idx],idx))
        df['dstname'][idx] = countries[countries['country3'] == df1.iloc[idx]]['name'].reset_index(drop=True)[0]
    else:
        tmp = tmp.reset_index(drop=True)[0]
    df['dstname'][idx] = tmp
    

    """
    TODO
    Cambia cordinadas de destino de acuerdo al nombre del dispositivo
    """
    tmp = Tec[Tec['Campus'] == df1.iloc[idx]]['longitud']
    if len(tmp) == 0:
        tmp = countries[countries['country3'] == df['dstloc'][idx]]['longitude'].reset_index(drop=True)[0]
    else:
        tmp = tmp.reset_index(drop=True)[0]
    
    df['dstlong'][idx] = tmp
    ################# consertar os splits########################l
    tmp = Tec[Tec['Campus'] == df1.iloc[idx]]['latitud']
    if len(tmp) != 0:
        tmp = tmp.reset_index(drop=True)[0]
    else:
        tmp = countries[countries['country3'] == df['dstloc'][idx]]['latitude'].reset_index(drop=True)[0]
    df['dstlat'][idx] = tmp    
    
    """
    Cambia cordinadas de fuente 
    """
    tmp = str(countries[countries['country'] == df['srcloc'][idx]]['longitude']).split()[1]
    df['srclong'][idx] = tmp
    tmp = str(countries[countries['country'] == df['srcloc'][idx]]['latitude']).split()[1]
    df['srclat'][idx] = tmp    
    

Campus no encontrado: CAN, indice=785
Campus no encontrado: CAN, indice=788
Campus no encontrado: CAN, indice=792
Pais no encontrado: alpha2=AP, indice=819
Pais no encontrado: alpha2=AP, indice=820
Pais no encontrado: alpha2=AP, indice=821
Pais no encontrado: alpha2=AP, indice=960
Pais no encontrado: alpha2=AP, indice=1020
Pais no encontrado: alpha2=EU, indice=1614
Campus no encontrado: CAN, indice=2145
Pais no encontrado: alpha2=EU, indice=2398
Campus no encontrado: CAN, indice=2514
Campus no encontrado: CAN, indice=2517
Campus no encontrado: CAN, indice=2520
Pais no encontrado: alpha2=AP, indice=2751
Campus no encontrado: CAN, indice=2955
Campus no encontrado: CAN, indice=3027
Campus no encontrado: CAN, indice=3253
Campus no encontrado: CAN, indice=3275
Campus no encontrado: CAN, indice=3282
Campus no encontrado: CAN, indice=3320
Campus no encontrado: CAN, indice=3346
Campus no encontrado: CAN, indice=3347
Campus no encontrado: CAN, indice=3349
Campus no encontrado: CAN, indice=3352


In [112]:
a = ['device_name',
 'direction',
 'src',
 'srclat',
 'srclong',
 'srcloc',
 'srcname',
 'dst',
 'dstlat',
 'dstlong',
 'dstloc',
 'dstname',
 'severity',
 'subtype',
 'threatid',
 'time_generated']

In [113]:
df = df[a]

In [142]:
df = df.sort_values('time_generated',ascending=False)

In [143]:
df.reset_index(drop=True,inplace=True)

In [144]:
df

,device_name,direction,src,srclat,srclong,srcloc,srcname,dst,dstlat,dstlong,dstloc,dstname,severity,subtype,threatid,time_generated
0,CP-CEDES-P1,client-to-server,189.56.79.66,-10.0,-55.0,BR,Brazil,10.2.18.231,25.6514,-100.29,CEDES,CEDES,orange,vulnerability,MAIL: User Login Brute Force Attempt,2018/09/27 17:18:23
1,CP-CEDES-P1,client-to-server,189.56.79.66,-10.0,-55.0,BR,Brazil,10.2.18.229,25.6514,-100.29,CEDES,CEDES,orange,vulnerability,MAIL: User Login Brute Force Attempt,2018/09/27 17:18:22
2,CP-CEDES-P1,client-to-server,189.56.79.66,-10.0,-55.0,BR,Brazil,10.2.18.230,25.6514,-100.29,CEDES,CEDES,orange,vulnerability,MAIL: User Login Brute Force Attempt,2018/09/27 17:18:19
3,CP-LEO-1,client-to-server,104.10.102.43,38.0,-97.0,US,United States,10.24.140.6,21.1663,-101.716,LEO,León,orange,vulnerability,Telnet Authentication Brute Force Attempt,2018/09/27 17:18:18
4,CP-HGO-1,client-to-server,196.188.0.86,8.0,38.0,ET,Ethiopia,10.24.30.5,20.0965,-98.7651,HGO,Hidalgo,orange,vulnerability,SMB DoublePulsar Ping Detection,2018/09/27 17:18:17
5,CP-CEDES-P1,client-to-server,189.56.79.66,-10.0,-55.0,BR,Brazil,10.2.18.230,25.6514,-100.29,CEDES,CEDES,orange,vulnerability,MAIL: User Login Brute Force Attempt,2018/09/27 17:18:17
6,CP-CEDES-P1,client-to-server,189.56.79.66,-10.0,-55.0,BR,Brazil,10.2.18.231,25.6514,-100.29,CEDES,CEDES,orange,vulnerability,MAIL: User Login Brute Force Attempt,2018/09/27 17:18:16
7,CP-CEDES-P1,client-to-server,189.56.79.66,-10.0,-55.0,BR,Brazil,10.2.18.229,25.6514,-100.29,CEDES,CEDES,orange,vulnerability,MAIL: User Login Brute Force Attempt,2018/09/27 17:18:13
8,UTM-FW-IPSEC,client-to-server,10.79.10.112,23.0,-102.0,MX,Mexico,10.73.0.93,25.744,-100.283,FW,Firewall San Nicolás,yellow,vulnerability,Microsoft Active Directory DCSync Attempt Dete...,2018/09/27 17:18:04
9,CP-CEDES-P1,client-to-server,189.56.79.66,-10.0,-55.0,BR,Brazil,10.2.18.230,25.6514,-100.29,CEDES,CEDES,orange,vulnerability,MAIL: User Login Brute Force Attempt,2018/09/27 17:18:03


In [145]:
df.to_json(os.path.expanduser('~/NorsePi/XML/_LastHour.json'),orient='index')

In [133]:
cat ~/NorsePi/XML/_LastHour.json | python -m json.tool

{
    "0": {
        "device_name": "CP-CEDES-P1",
        "direction": "client-to-server",
        "src": "173.199.232.106",
        "srclat": "38.0",
        "srclong": "-97.0",
        "srcloc": "US",
        "srcname": "United States",
        "dst": "10.2.18.231",
        "dstlat": 25.65142,
        "dstlong": -100.290328,
        "dstloc": "CEDES",
        "dstname": "CEDES",
        "severity": "orange",
        "subtype": "vulnerability",
        "threatid": "MAIL: User Login Brute Force Attempt",
        "time_generated": "2018/09/27 12:49:07"
    },
    "1": {
        "device_name": "CP-CEDES-P1",
        "direction": "client-to-server",
        "src": "189.56.79.66",
        "srclat": "-10.0",
        "srclong": "-55.0",
        "srcloc": "BR",
        "srcname": "Brazil",
        "dst": "10.2.18.231",
        "dstlat": 25.65142,
        "dstlong": -100.290328,
        "dstloc": "CEDES",
        "dstname": "CEDES",
        "severity": "orange",
        "subtype": "vulnerabil

        "srclat": "23.0",
        "srclong": "-102.0",
        "srcloc": "MX",
        "srcname": "Mexico",
        "dst": "38.122.71.154",
        "dstlat": 19.020044,
        "dstlong": -98.244838,
        "dstloc": "PUE",
        "dstname": "Puebla",
        "severity": "orange",
        "subtype": "vulnerability",
        "threatid": "Exim Dovecot Mail Address Handling Command Injection Vulnerability",
        "time_generated": "2018/09/27 13:02:58"
    },
    "390": {
        "device_name": "CP-CEDES-P1",
        "direction": "client-to-server",
        "src": "189.56.79.66",
        "srclat": "-10.0",
        "srclong": "-55.0",
        "srcloc": "BR",
        "srcname": "Brazil",
        "dst": "10.2.18.230",
        "dstlat": 25.65142,
        "dstlong": -100.290328,
        "dstloc": "CEDES",
        "dstname": "CEDES",
        "severity": "orange",
        "subtype": "vulnerability",
        "threatid": "MAIL: User Login Brute Force Attempt",
        "time_generated": "2018/0

        "srclat": "46.0",
        "srclong": "2.0",
        "srcloc": "FR",
        "srcname": "France",
        "dst": "10.2.18.239",
        "dstlat": 25.65142,
        "dstlong": -100.290328,
        "dstloc": "CEDES",
        "dstname": "CEDES",
        "severity": "orange",
        "subtype": "vulnerability",
        "threatid": "MAIL: User Login Brute Force Attempt",
        "time_generated": "2018/09/27 14:03:42"
    },
    "1182": {
        "device_name": "CP-CEDES-P1",
        "direction": "client-to-server",
        "src": "217.61.98.124",
        "srclat": "46.0",
        "srclong": "2.0",
        "srcloc": "FR",
        "srcname": "France",
        "dst": "10.2.18.239",
        "dstlat": 25.65142,
        "dstlong": -100.290328,
        "dstloc": "CEDES",
        "dstname": "CEDES",
        "severity": "orange",
        "subtype": "vulnerability",
        "threatid": "MAIL: User Login Brute Force Attempt",
        "time_generated": "2018/09/27 14:03:49"
    },
    "1183": {

        "dst": "10.2.18.230",
        "dstlat": 25.65142,
        "dstlong": -100.290328,
        "dstloc": "CEDES",
        "dstname": "CEDES",
        "severity": "orange",
        "subtype": "vulnerability",
        "threatid": "MAIL: User Login Brute Force Attempt",
        "time_generated": "2018/09/27 15:25:04"
    },
    "1938": {
        "device_name": "CP-CEDES-P1",
        "direction": "client-to-server",
        "src": "189.56.79.66",
        "srclat": "-10.0",
        "srclong": "-55.0",
        "srcloc": "BR",
        "srcname": "Brazil",
        "dst": "10.2.18.231",
        "dstlat": 25.65142,
        "dstlong": -100.290328,
        "dstloc": "CEDES",
        "dstname": "CEDES",
        "severity": "orange",
        "subtype": "vulnerability",
        "threatid": "MAIL: User Login Brute Force Attempt",
        "time_generated": "2018/09/27 15:25:04"
    },
    "1939": {
        "device_name": "CP-CEDES-P1",
        "direction": "client-to-server",
        "src": "217.61.

        "device_name": "CP-CEDES-P1",
        "direction": "client-to-server",
        "src": "189.56.79.66",
        "srclat": "-10.0",
        "srclong": "-55.0",
        "srcloc": "BR",
        "srcname": "Brazil",
        "dst": "10.2.18.231",
        "dstlat": 25.65142,
        "dstlong": -100.290328,
        "dstloc": "CEDES",
        "dstname": "CEDES",
        "severity": "orange",
        "subtype": "vulnerability",
        "threatid": "MAIL: User Login Brute Force Attempt",
        "time_generated": "2018/09/27 15:52:22"
    },
    "2651": {
        "device_name": "CP-CEDES-P1",
        "direction": "client-to-server",
        "src": "189.56.79.66",
        "srclat": "-10.0",
        "srclong": "-55.0",
        "srcloc": "BR",
        "srcname": "Brazil",
        "dst": "10.2.18.230",
        "dstlat": 25.65142,
        "dstlong": -100.290328,
        "dstloc": "CEDES",
        "dstname": "CEDES",
        "severity": "orange",
        "subtype": "vulnerability",
        "thre

        "threatid": "MAIL: User Login Brute Force Attempt",
        "time_generated": "2018/09/27 16:09:40"
    },
    "3140": {
        "device_name": "CP-TOL-1",
        "direction": "client-to-server",
        "src": "59.33.53.55",
        "srclat": "35.0",
        "srclong": "105.0",
        "srcloc": "CN",
        "srcname": "China",
        "dst": "10.25.163.47",
        "dstlat": 19.268825,
        "dstlong": -99.705974,
        "dstloc": "TOL",
        "dstname": "Toluca",
        "severity": "orange",
        "subtype": "vulnerability",
        "threatid": "Microsoft SQL Server User Authentication Brute Force Attempt",
        "time_generated": "2018/09/27 16:09:43"
    },
    "3141": {
        "device_name": "CP-LAG-1",
        "direction": "client-to-server",
        "src": "190.79.152.201",
        "srclat": "8.0",
        "srclong": "-66.0",
        "srcloc": "VE",
        "srcname": "Venezuela, Bolivarian Republic of",
        "dst": "10.32.101.44",
        "dstlat": 25.5

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

